In [3]:
pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (93.5 kB/s)
Reading package li

In [32]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141804 sha256=83fe25febb5d1d67405f49568f181683406f2ddcb0def37eac6d6e56447821fd
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [1]:
## 라이브러리 설치
import pandas as pd
import numpy as np
import os
import datetime as datetime
import re
import konlpy as konp 
from konlpy.tag import Okt
from konlpy.tag import Mecab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. 뉴스 데이터 ESG 감성분석

In [3]:
df_news_actv = pd.read_csv('/content/drive/MyDrive/넘블 나이스 디앤비 챌린지/2. ESG raw data/news_actv_pre.csv')
df_news_clos = pd.read_csv('/content/drive/MyDrive/넘블 나이스 디앤비 챌린지/2. ESG raw data/news_clos_pre.csv')

In [4]:
# 불필요열 삭제
df_news_actv = df_news_actv.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
df_news_clos = df_news_clos.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)

# 날짜 기준 인덱싱
df_news_actv = df_news_actv[df_news_actv['pubDate'].str.contains('2018|2019|2020|2021')]
df_news_actv = df_news_actv[~df_news_actv['pubDate'].str.contains('▲ 낙찰업체')]

df_news_clos = df_news_clos[df_news_clos['pubDate'].str.contains('2018ㅁ|2019|2020|2021')]
df_news_clos = df_news_clos[~df_news_clos['pubDate'].str.contains('▲ 낙찰업체')]

# 전체 텍스트 생성
df_news_actv['full_text'] = df_news_actv['title'] + df_news_actv['description'] 
df_news_clos['full_text'] = df_news_clos['title'] + df_news_clos['description'] 

In [5]:
## 크롤링 한 데이터를 기반으로 심리지수 산출

#E - 환경
centor_e = ['친환경','환경친화','환경친화','환경혁신','환경오염','친환경기술','환경문제',
'기후','기후변화','기후위기','온실가스',
'생물다양성','생물에너지','생태계',
'공해','탄소배출','탄소발자국','탄소중립','지구온난화','온실가스','공해저감','천연가스','탄소제로','탄소저감','저감',
'지속성','지속가능','방출',
'신재생','재생에너지','신재생에너지','재생소재','재사용','재생가능','에너지효율','그린에너지','그린 빌딩','청정','청정기술','업사이클링','재활용',
'태양열','풍력','바이오 연료']

#S - 인적 자본
centor_s = ['다양성','상생','사회','안전','노동','평등','CSR','기부','지역',
            '지역사회','사생활','안전','건강','근로','조건','근로 환경','분위기',
            '사업장','관행','품질','사생활','주주','근로자','직원','작업 환경','봉사']

#G - 거버넌스
centor_g = ['권한','이사회','자금','예산','성과급','예산 유연성','지배구조','경영진',
            'CEO','이의제기','임원진','위험관리','리스크관리','정책 유연성','규제','구조조정','이해관계자','투명성','사면']


In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [20]:
def get_ngrams(s, n):
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', ' ', s)

    # Break sentence in the token, remove empty tokens
    text = []
    tokens = mecab.nouns(s)

    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]


E카운트 = []
S카운트 = []
G카운트 = []

def ESG_Calculator(text):
    '''
    해당 기업의 ESG 스코어를 산출 중입니다...
    '''

    try:      
        text=re.sub('[\\n]','',text)
        text=get_ngrams(text,1)
        counts=[]
        counts= [text.count(x) for x in centor_e]
        e_Freq=sum(counts)
        E카운트.append(e_Freq)
        counts= [text.count(x) for x in centor_s]
        s_Freq=sum(counts) 
        S카운트.append(s_Freq)
        counts= [text.count(x) for x in centor_g]
        g_Freq=sum(counts) 
        G카운트.append(g_Freq) 
        
    except Exception as ex:
          excect_escape=0 
          E카운트.append(excect_escape)
          S카운트.append(excect_escape)
          G카운트.append(excect_escape)

lists = df_news_clos['full_text'].to_list()

for text in lists :
  ESG_Calculator(text)
  
  esg_df = pd.DataFrame({'E카운트': E카운트, 'S카운트': S카운트, 'G카운트': E카운트})


In [21]:
esg_df.to_csv('/content/drive/MyDrive/넘블 나이스 디앤비 챌린지/2. ESG raw data/ESG_news_clos.csv')

In [ ]:
df_news_actv = df_news_actv[df_news_actv['pubDate'].str.contains('2018|2019|2020|2021')]
df_news_actv = df_news_actv[~df_news_actv['pubDate'].str.contains('▲ 낙찰업체')]